In [ ]:
import openai
client = openai.OpenAI()

# ai agent에게 내가 어떤 함수를 가지고 있는지, 그 함수들이 어떻게 생긴건지, 이름은 뭔지, 어떻게 호출하는지 알려줘야함
# 그리고 ai에게 내가 어떤 함수를 호출해야하는지 알려줘~ 이렇게 부탁함.. 오호.. 한글도 잘되는군아
# 그럼 si에서는 뭔 함수를 호출해야하는지 알려주는 거임 끝! 그게 ai에이전트임..
PROMPT = """
I have the following functions in my system. 

`get_weather`
`get_currency`
`get_news`

이 모든 함수들은 나라 이름을 인자로 받습니다. (i.e get_news("USA"))

자 이제 내가 실행해야 할 함수의 이름을 알려주세요

그 외에는 아무 말도 하지 마세요. 단지 함수 이름과 인자만 말해주세요.

다음 질문에 대답해주세요:

한국의 날씨는 어떤가요?

"""

# ChatCompletion 객체를 받게됨
#  client.chat.completions.create => openai에서 받은 응답을 파이썬에서쓸수 있게 변환해줌 
#  이 모델들은 그냥 확률로 계산을 한다고함.
response = client.chat.completions.create(
    model ="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": PROMPT  # 메시지 내용
        }
    ],
    # n=10 -> 초이스 10개
)

response

# 이걸 먼저 실행 시킨후에 아래 message 출력 시키면 됨
# 'get_weather("Korea")' 이렇게 나옴 굳~ 

ChatCompletion(id='chatcmpl-CzHTREjYL45qzowza5rYVGz2otV8E', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='get_weather("Korea")', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768721213, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_8bbc38b4db', usage=CompletionUsage(completion_tokens=6, prompt_tokens=101, total_tokens=107, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
# for choice in response.choices:
#     print(choice)

message = response.choices[0].message.content
message
# 'The capital of Greece is Athens.' 으로 메시지 내용 나옴 신기하군..
    

'get_weather("Korea")'

In [40]:
import openai , json

client = openai.OpenAI()
messages = []


In [ ]:
def get_weather(city):
    return "33 degress celcius"

# 기능을 더 넣고싶다면 여기에 그냥 함수들 넣으면됨! 

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [42]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters":{
                "type": "object",
                "properties":{
                    "city":{
                        "type":"string",
                        "description" : "The name of the city to get the weather of."
                    }
                },
                "required":["city"]
            }
        }
    }
]

In [43]:
from openai.types.chat import ChatCompletionMessage

# 단지 기록만 남겨놓는거임
# process_ai_response = ai한테 받은 메시지 안에 tool_call이 있는 확인하고, 그요청을 메모리에 저장하기
def process_ai_response(message:ChatCompletionMessage ):
    if message.tool_calls:  # tool_calls가 하나라도 있으면 ai가 실행요청한 걸 메시지 메모리에 저장
        messages.append({
            "role":"assistant",
            "content":message.content or "",
            "tool_calls":[{
                "id": tool_call.id,
                "type":"function",
                "function":{
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                }
            } for tool_call in message.tool_calls]
        })

        # ai가 실행하려는 각 tool_call마다 함수이름과 arguments를 뽑아내고 
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name  # 콤마 제거! 튜플이 되지 않도록
            arguments = tool_call.function.arguments # 걍 문자열  {"city":"korea"}

            print(f"Calling function : {function_name} with {arguments}")

            # 그 arguments 문자열을 python 딕셔너리로 바꾸고
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            # ai가 요청한 함수 이름으로 function_map에서 함수를 가져옴 =여기서 이름을 보내주니까 
            function_to_run = FUNCTION_MAP.get(function_name) 

            result = function_to_run(**arguments)

            print(f"Run {function_name} with arg {arguments} for a result of {result} ")

            # messages 메모리에 또 하나의 메시지 추가 
            # 아래에 맞게 넣는거다~ 
            messages.append({
                "role":"tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content":result
            })

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI : {message.content}")




# 쳇봇 만든거임 이거
# 입력한 메모리를 가지고 있게되었음 얘는 
 # 메시지 메모리와 TOOLS(구조)를 함께 전달해서 ai호출
def call_ai():
    response = client.chat.completions.create(
        model ="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)
    print(response)
    message = response.choices[0].message.content
    print(f"User: ${message}")

In [44]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == 'q':
        break
    else:
        messages.append({
            "role":"user",
            "content": message
        })
        print(f"User: ${message}")
        call_ai()
       

User: $내이름은 소망이야
AI : 안녕하세요, 소망님! 어떻게 도와드릴까요?
ChatCompletion(id='chatcmpl-CzHEk8FNt1g2BpVVVKHjIBGrsIOjH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 소망님! 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1768720302, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_29330a9688', usage=CompletionUsage(completion_tokens=15, prompt_tokens=67, total_tokens=82, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
User: $안녕하세요, 소망님! 어떻게 도와드릴까요?
User: $내이름이 뭐라고?
AI : 소망님이라고 하셨습니다! 맞나요?
ChatCompletion(id='chatcmpl-CzHEp738Z19eju3LUVpOiAR9Yh0tt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(co

In [27]:
a = '{"city":"korea"}'
b = json.loads(a)
# b['city']
# **b # => city='korea'
get_weather(city='Korea')

'33 degress celcius'

In [45]:
messages

[{'role': 'user', 'content': '내이름은 소망이야'},
 {'role': 'assistant', 'content': '안녕하세요, 소망님! 어떻게 도와드릴까요?'},
 {'role': 'user', 'content': '내이름이 뭐라고?'},
 {'role': 'assistant', 'content': '소망님이라고 하셨습니다! 맞나요?'},
 {'role': 'user', 'content': '오늘 한국의 날씨는 어댸'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_qwtFcI9bwJnjO1NqISfSk8vt',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_qwtFcI9bwJnjO1NqISfSk8vt',
  'name': 'get_weather',
  'content': '33 degress celcius'},
 {'role': 'assistant',
  'content': '오늘 서울의 날씨는 33도입니다. 더운 날씨에 주의하세요! 다른 도움이 필요하시면 말씀해 주세요.'}]